In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data_loc = 'Rent-tables-June-2023-quarter.xlsx'
df = pd.read_excel(data_loc, sheet_name="Postcode")

In [6]:
df.head()

,Postcode,Dwelling Types,Number of Bedrooms,First Quartile Weekly Rent for New Bonds\n$,Median Weekly Rent for New Bonds\n$,Third Quartile Weekly Rent for New Bonds\n$,New Bonds Lodged\nNo.,Total Bonds Held\nNo.,Quarterly change in Median Weekly Rent,Annual change in Median Weekly Rent,Quarterly change in New Bonds Lodged,Annual change in New Bonds Lodged
0,2000,Total,Total,617,770,1050,"1,656","11,203",2.67%,10.00%,-10.49%,30.19%
1,2000,Total,Bedsitter,528,610,676,58,437,1.67%,35.56%,56.76%,28.89%
2,2000,Total,1 Bedroom,559,680,800,"1,052","5,476",1.64%,8.80%,.19%,53.13%
3,2000,Total,Not Specified,619,669,945,s,460,-29.58%,7.04%,-94.93%,-29.17%
4,2000,Total,2 Bedrooms,1000,1150,1300,444,"4,110",-4.17%,27.78%,25.42%,2.07%


# Data Processing

In [7]:
clean_df = df[['Postcode', 'Number of Bedrooms', 'First Quartile Weekly Rent for New Bonds\n$', 'Median Weekly Rent for New Bonds\n$', 'Third Quartile Weekly Rent for New Bonds\n$' ]]

column_mapping = {
    'Postcode': 'PostalCode',
    'Number of Bedrooms': 'Bedrooms',
    'First Quartile Weekly Rent for New Bonds\n$': 'CheaperRent',
    'Median Weekly Rent for New Bonds\n$': 'MedianRent',
    'Third Quartile Weekly Rent for New Bonds\n$': 'HigherRent'
}
clean_df = clean_df.rename(columns = column_mapping)
clean_df

,PostalCode,Bedrooms,CheaperRent,MedianRent,HigherRent
0,2000,Total,617,770,1050
1,2000,Bedsitter,528,610,676
2,2000,1 Bedroom,559,680,800
3,2000,Not Specified,619,669,945
4,2000,2 Bedrooms,1000,1150,1300
...,...,...,...,...,...
16279,2900,1 Bedroom,-,-,-
16280,2905,Total,-,-,-
16281,2905,4 or more Bedrooms,-,-,-
16282,2905,Total,-,-,-


In [8]:
clean_df = clean_df[clean_df['Bedrooms'] != 'Not Specified']
clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('4 or more Bedrooms', 4)
clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('1 Bedroom', 1)
clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('2 Bedrooms', 2)
clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('3 Bedrooms', 3)
clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('Bedsitter', 0)
clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('Total', -1)

clean_df = clean_df.dropna()


C:\Users\akshay\AppData\Local\Temp\ipykernel_14388\448352224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('4 or more Bedrooms', 4)
C:\Users\akshay\AppData\Local\Temp\ipykernel_14388\448352224.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['Bedrooms'] = clean_df['Bedrooms'].replace('1 Bedroom', 1)
C:\Users\akshay\AppData\Local\Temp\ipykernel_14388\448352224.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [9]:
clean_df = clean_df.loc[~(clean_df[['CheaperRent', 'MedianRent', 'HigherRent']] == '-').any(axis=1)]
clean_df = clean_df.reset_index(drop=True)

In [10]:

clean_df['CheaperRent'] = clean_df['CheaperRent'].astype(int)
clean_df['MedianRent'] = clean_df['MedianRent'].astype(int)
clean_df['HigherRent'] = clean_df['HigherRent'].astype(int)


# Processing Postcodes

In [77]:


post_codes = 'australian_postcodes.csv'
pc_df = pd.read_csv(post_codes)
pc_df = pc_df[pc_df['state'] == 'NSW']
pc_df = pc_df[pc_df['type'] == 'Delivery Area']
pc_df = pc_df[['postcode', 'locality', 'long', 'lat']]


pc_df = pc_df.rename(columns = {'postcode': 'PostalCode', 'locality': 'Locality'})
pc_df_file = 'pc_df.csv'
pc_df.to_csv(pc_df_file, index=False) 
pc_df


,PostalCode,Locality,long,lat
715,2000,BARANGAROO,151.201580,-33.860520
717,2000,DAWES POINT,151.256649,-33.859953
718,2000,HAYMARKET,151.256649,-33.859953
719,2000,MILLERS POINT,151.256649,-33.859953
720,2000,PARLIAMENT HOUSE,151.256649,-33.859953
...,...,...,...,...
11011,4380,MINGOOLA,151.529213,-28.940993
11018,4380,RUBY CREEK,152.018346,-28.625911
11025,4380,UNDERCLIFFE,152.182263,-28.622551
11034,4383,JENNINGS,151.969412,-28.940512


In [18]:
def filter_by_postcode(postCode, df):
    postCode = int(postCode)
    post_view = df[df['PostalCode'] == postCode]
    return post_view


def filter_by_bedrooms(bedrooms, df):
    bedrooms = int(bedrooms)
    post_view = df[df['Bedrooms'] == bedrooms]
    return post_view


'''
   Given a postcode, #bedrooms, return expected rent
'''
def average_rent(postCode, bedrooms, df):
    compute_df = filter_by_postcode(postCode, df)
    compute_df = filter_by_bedrooms(bedrooms, compute_df)
    means = compute_df.mean().astype(int)
    return means


def median_rent_locality(locality, bedrooms, df):
    postcode = get_postcode(locality)
    compute_df = filter_by_postcode(postcode, df)
    compute_df = filter_by_bedrooms(bedrooms, compute_df)
    return compute_df['MedianRent'].mean()

def get_median_rent_post(postcode, bedrooms, df):
    compute_df = filter_by_postcode(postcode, df)
    compute_df = filter_by_bedrooms(bedrooms, compute_df)
    return compute_df['MedianRent'].mean()


In [19]:
def get_postcode(locality):
    locality = locality.upper()
    result = pc_df[pc_df['locality'] == locality]
    if not result.empty:
        return result.iloc[0]['PostalCode']
    else:
        return None
    
def get_locality(postcode):
    result = pc_df[pc_df['PostalCode'] == postcode]
    if not result.empty:
        return result.iloc[0]['locality']
    else:
        return None
    
    
def df_median_rent(df, bedrooms):
    result_df = pd.DataFrame(columns=['PostalCode', 'MedianRent', 'bedrooms'])
    unique_postcodes = df['PostalCode'].unique()

    for postcode in unique_postcodes:
        subset_df = df[df['PostalCode'] == postcode]
        subset_df = subset_df[subset_df['Bedrooms'] == bedrooms]
        median_rent = get_median_rent_post(postcode, bedrooms, subset_df)
        result_df = pd.concat([result_df, pd.DataFrame({'PostalCode': [postcode], 'MedianRent': [median_rent], 'bedrooms': [bedrooms]})], ignore_index=True)
        result_df = result_df.dropna()
    return result_df
        
    
def get_overview_rent(df):
    result_df = pd.DataFrame(columns=['PostalCode', 'MedianRent', 'bedrooms'])
    unique_postcodes = df['PostalCode'].unique()

    for postcode in unique_postcodes:
        subset_df = df[df['PostalCode'] == postcode]
        median_rent = get_overview_rent_post(postcode, subset_df)
        result_df = pd.concat([result_df, pd.DataFrame({'PostalCode': [postcode], 'MedianRent': [median_rent], 'bedrooms': [bedrooms]})], ignore_index=True)
        result_df = result_df.dropna()
    return result_df
    
    
def assign_affordability(df, value):
    df_copy = df.copy()
    
    def assign_colour(median_val, value):
        if median_val >= 1.35 * value:
            return 5
        elif 1.2 * value <= median_val < 1.35 * value:
            return 4
        elif 0.8 * value <= median_val < 1.2 * value:
            return 3
        elif 0.75 * value <= median_val < 0.8 * value:
            return 2
        else:
            return 1
    
    affordabilities = []

    for median_val in df_copy['MedianRent']:
        affordabilities.append(assign_colour(median_val, value))
    
    df_copy['Affordability'] = affordabilities
    return df_copy

def get_affordable_posts(df, afford_rating):
    result_df = df[df['Affordability'] <= afford_rating]
    return result_df
    

In [20]:
median_rent_data = df_median_rent(clean_df, 3)
df_computed = assign_affordability(median_rent_data, 900)
result_df = get_affordable_posts(df_computed, 3)
result_df

,PostalCode,MedianRent,bedrooms,Affordability
7,2016,1058.333333,3,3
31,2040,1025.0,3,3
33,2042,1059.333333,3,3
35,2044,970.0,3,3
36,2046,941.666667,3,3
...,...,...,...,...
354,2835,275.0,3,1
355,2850,492.5,3,1
356,2870,380.0,3,1
357,2871,372.5,3,1


In [24]:
def add_locality(df, post_code_data):
    df_copy = df.copy()
    merged_df = pd.merge(df_copy, post_code_data, on='PostalCode', how='left')
    localities_grouped = merged_df.groupby('PostalCode')['Locality'].apply(lambda x: ', '.join(x.dropna())).reset_index()
    df_copy = pd.merge(df_copy, localities_grouped, on='PostalCode', how='left')
    df_copy = df_copy.rename(columns={'Locality': 'Localities'})
    df_copy = pd.merge(df_copy, post_code_data[['PostalCode', 'lat', 'long']], on='PostalCode', how='left')
    return df_copy


df_with_local = add_locality(result_df, pc_df)
df_with_local

,PostalCode,MedianRent,bedrooms,Affordability,Localities,lat,long
0,2016,1058.333333,3,3,REDFERN,-33.894912,151.206211
1,2040,1025.0,3,3,"LEICHHARDT, LILYFIELD",-33.878774,151.156819
2,2040,1025.0,3,3,"LEICHHARDT, LILYFIELD",-33.878774,151.156819
3,2042,1059.333333,3,3,"ENMORE, NEWTOWN",-33.900649,151.175354
4,2042,1059.333333,3,3,"ENMORE, NEWTOWN",-33.900649,151.175354
...,...,...,...,...,...,...,...
3243,2880,320.0,3,1,"BROKEN HILL, BROKEN HILL NORTH, BROKEN HILL WE...",-30.441973,142.170513
3244,2880,320.0,3,1,"BROKEN HILL, BROKEN HILL NORTH, BROKEN HILL WE...",-30.170441,142.203381
3245,2880,320.0,3,1,"BROKEN HILL, BROKEN HILL NORTH, BROKEN HILL WE...",-30.170441,142.203381
3246,2880,320.0,3,1,"BROKEN HILL, BROKEN HILL NORTH, BROKEN HILL WE...",-30.170441,142.203381


In [22]:
pc_df

,PostalCode,Locality,long,lat
715,2000,BARANGAROO,151.201580,-33.860520
717,2000,DAWES POINT,151.256649,-33.859953
718,2000,HAYMARKET,151.256649,-33.859953
719,2000,MILLERS POINT,151.256649,-33.859953
720,2000,PARLIAMENT HOUSE,151.256649,-33.859953
...,...,...,...,...
11011,4380,MINGOOLA,151.529213,-28.940993
11018,4380,RUBY CREEK,152.018346,-28.625911
11025,4380,UNDERCLIFFE,152.182263,-28.622551
11034,4383,JENNINGS,151.969412,-28.940512


# EDA

In [146]:
clean_df

,PostalCode,Bedrooms,CheaperRent,MedianRent,HigherRent
0,2000,-1,617,770,1050
1,2000,0,528,610,676
2,2000,1,559,680,800
3,2000,2,1000,1150,1300
4,2000,3,1425,1695,2050
...,...,...,...,...,...
3937,2880,2,250,285,295
3938,2880,3,277,320,380
3939,2880,-1,190,235,288
3940,2880,1,190,190,250


In [74]:
pc_df

,PostalCode,Locality
715,2000,BARANGAROO
717,2000,DAWES POINT
718,2000,HAYMARKET
719,2000,MILLERS POINT
720,2000,PARLIAMENT HOUSE
...,...,...
11011,4380,MINGOOLA
11018,4380,RUBY CREEK
11025,4380,UNDERCLIFFE
11034,4383,JENNINGS
